# DATA PRE-PROCESSING

### Data Import via Yahoo finance, Alpha Vantage, Tiingo

In [ ]:
%pip install yahoo_fin
%pip install requests_html
%pip install alpha-vantage

In [187]:
import yfinance as yf
import yahoo_fin.stock_info as si
import pandas as pd
from datetime import datetime
import numpy as np



### OHLC, Volume, Corporate Actions data from multiple stickers

###### Data Collection

In [182]:
def get_price_data(ticker, start_date, end_date):
    """Get raw OHLCV data and format it into the desired DataFrame structure."""
    df = yf.download(ticker, start=start_date, end=end_date)
    # Reset index to turn Date into a column
    df = df.reset_index()
    # Flatten the MultiIndex columns (assuming columns have three levels)
    # Extract the metric names (second level of MultiIndex)
    new_columns = []
    for col in df.columns:
        if col == 'Date':
            new_columns.append('Date')
        else:
            # Get the second level of the MultiIndex (e.g., 'Close', 'High')
            new_columns.append(col[1])
    df.columns = new_columns
    # Add ticker column
    df['ticker'] = ticker
    df.columns.values[0] = "Date"
    df.columns.values[1] = "Close"
    df.columns.values[2] = "High"
    df.columns.values[3] = "Low"
    df.columns.values[4] = "Open"
    df.columns.values[5] = "Volume"

    # Reorder columns to the desired order
    return df
# Test the function
test_df = get_price_data("AAPL", "2023-01-01", "2023-01-10")
print("\nProcessed Price Data:\n")
test_df.head()

[*********************100%***********************]  1 of 1 completed


Processed Price Data:



,Date,Close,High,Low,Open,Volume,ticker
0,2023-01-03,123.632523,129.395510,122.742865,128.782641,112117500,AAPL
1,2023-01-04,124.907707,127.181276,123.642420,125.431615,89113600,AAPL
2,2023-01-05,123.583107,126.301500,123.326101,125.668857,80962700,AAPL
3,2023-01-06,128.130203,128.792501,123.454572,124.561702,87754700,AAPL
4,2023-01-09,128.654129,131.876670,128.397123,128.970458,70790800,AAPL


## Feature Engineering

#### 1. Calculate Trend of Moving Averages

In [184]:
def add_trend_indicators(df):
    """Add SMA, EMA, MACD (grouped by ticker)"""
    # Ensure we group by ticker if multiple stocks exist
    grouped = df.groupby('ticker', group_keys=False)

    # Calculate indicators per ticker
    def calculate_indicators(group):
        # Moving Averages
        group['SMA_20'] = group['Close'].rolling(window=20).mean()
        group['EMA_20'] = group['Close'].ewm(span=20, adjust=False).mean()

        # MACD
        exp12 = group['Close'].ewm(span=12, adjust=False).mean()
        exp26 = group['Close'].ewm(span=26, adjust=False).mean()
        group['MACD'] = exp12 - exp26
        group['Signal_Line'] = group['MACD'].ewm(span=9, adjust=False).mean()
        return group

    return grouped.apply(calculate_indicators)
# Get 1 month of data for proper SMA calculation
test_df = get_price_data("AAPL", "2023-01-01", "2023-02-10")
trend_df = add_trend_indicators(test_df.copy())

# Show last 3 rows where SMA_20 exists
print("\nTrend Indicators:\n", trend_df[['Date', 'Close', 'SMA_20', 'EMA_20', 'MACD']].tail(3))

[*********************100%***********************]  1 of 1 completed


Trend Indicators:
          Date       Close      SMA_20      EMA_20      MACD
24 2023-02-07  152.872559  140.194459  141.334063  6.125100
25 2023-02-08  150.173950  141.241782  142.175957  6.064972
26 2023-02-09  149.135986  142.100793  142.838817  5.865945



<ipython-input-184-16f7ab0c4e8e>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grouped.apply(calculate_indicators)


#### 2. Momentum Indicators

In [186]:
def add_momentum_indicators(df):
    """Add RSI and Stochastic Oscillator"""
    # RSI
    delta = df['Close'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    avg_gain = gain.rolling(14).mean()
    avg_loss = loss.rolling(14).mean()
    rs = avg_gain / avg_loss
    df['RSI'] = 100 - (100 / (1 + rs))

    # Stochastic
    low14 = df['Low'].rolling(14).min()
    high14 = df['High'].rolling(14).max()
    df['%K'] = 100 * ((df['Close'] - low14) / (high14 - low14))
    df['%D'] = df['%K'].rolling(3).mean()
    return df

# Test
momentum_df = add_momentum_indicators(trend_df.copy())
print("\nMomentum Indicators:\n", momentum_df[['Date', 'RSI', '%K', '%D']].tail(3))


Momentum Indicators:
          Date        RSI         %K         %D
24 2023-02-07  80.205161  88.437118  84.102776
25 2023-02-08  73.888170  76.424960  80.310510
26 2023-02-09  69.519540  66.581105  77.147727


#### 3. Volatility Indicators

In [188]:
def add_volatility_indicators(df):
    """Add Bollinger Bands and ATR"""
    # Bollinger Bands
    df['Middle_Band'] = df['Close'].rolling(20).mean()
    df['Upper_Band'] = df['Middle_Band'] + 2*df['Close'].rolling(20).std()
    df['Lower_Band'] = df['Middle_Band'] - 2*df['Close'].rolling(20).std()

    # ATR
    high_low = df['High'] - df['Low']
    high_close = np.abs(df['High'] - df['Close'].shift())
    low_close = np.abs(df['Low'] - df['Close'].shift())
    tr = pd.concat([high_low, high_close, low_close], axis=1).max(axis=1)
    df['ATR'] = tr.rolling(14).mean()
    return df

# Test
volatility_df = add_volatility_indicators(momentum_df.copy())
print("\nVolatility Indicators:\n", volatility_df[['Date', 'Upper_Band', 'Lower_Band', 'ATR']].tail(3))


Volatility Indicators:
          Date  Upper_Band  Lower_Band       ATR
24 2023-02-07  154.518024  125.870893  4.158079
25 2023-02-08  155.248620  127.234943  4.228686
26 2023-02-09  155.813943  128.387643  4.236453


#### 4. Volume Indicators

In [189]:
def add_volume_indicators(df):
    """Add CMF and Volume ROC"""
    # Chaikin Money Flow
    mfm = ((df['Close'] - df['Low']) - (df['High'] - df['Close'])) / (df['High'] - df['Low'])
    mfm = mfm.replace([np.inf, -np.inf], 0)
    mfv = mfm * df['Volume']
    df['CMF'] = mfv.rolling(20).sum() / df['Volume'].rolling(20).sum()

    # Volume ROC
    df['Volume_ROC'] = df['Volume'].pct_change(1) * 100
    return df

# Test
volume_df = add_volume_indicators(volatility_df.copy())
print("\nVolume Indicators:\n", volume_df[['Date', 'CMF', 'Volume_ROC']].tail(3))


Volume Indicators:
          Date       CMF  Volume_ROC
24 2023-02-07  0.423644   19.273730
25 2023-02-08  0.371651  -23.045968
26 2023-02-09  0.300219  -12.652819


#### 5. Returns & Risk Metrics

In [190]:
def add_risk_metrics(df, risk_free_rate=0.0135):
    """Add returns and risk metrics"""
    # Returns
    df['Log_Returns'] = np.log(df['Close'] / df['Close'].shift(1))
    df['Rolling_Vol_21D'] = df['Log_Returns'].rolling(21).std() * np.sqrt(252)

    # Drawdown
    cumulative_returns = (1 + df['Log_Returns']).cumprod()
    peak = cumulative_returns.expanding().max()
    df['Drawdown'] = (peak - cumulative_returns) / peak

    # Sharpe Ratio
    df['Sharpe'] = (df['Log_Returns'].mean() - risk_free_rate/252) / df['Log_Returns'].std()
    return df

# Test
final_df = add_risk_metrics(volume_df.copy())
print("\nRisk Metrics:\n", final_df[['Date', 'Log_Returns', 'Rolling_Vol_21D']].tail(3))


Risk Metrics:
          Date  Log_Returns  Rolling_Vol_21D
24 2023-02-07     0.019062         0.217971
25 2023-02-08    -0.017810         0.235900
26 2023-02-09    -0.006936         0.240916


#### 6. Corporate Actions

In [191]:
def get_corporate_actions(ticker, start_date, end_date):
    """Get dividends and splits"""
    yf_ticker = yf.Ticker(ticker)
    return {
        'Dividends': yf_ticker.dividends.loc[start_date:end_date].sum(),
        'Splits': yf_ticker.splits.loc[start_date:end_date].count()
    }

# Test
print("\nCorporate Actions:", get_corporate_actions("AAPL", "2020-01-01", "2023-01-01"))


Corporate Actions: {'Dividends': 2.5825, 'Splits': 1}


### Incorporate all Features and return combined Dataframe

In [194]:
from concurrent.futures import process
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

start_date = "2020-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')

dow_list = si.tickers_dow()
def build_full_dataset(tickers, start_date, end_date):
    final_data = []

    for ticker in tickers:
        try:
            # Core data
            df = get_price_data(ticker, start_date, end_date)

            # Add features
            df = add_trend_indicators(df)
            df = add_momentum_indicators(df)
            df = add_volatility_indicators(df)
            df = add_volume_indicators(df)
            df = add_risk_metrics(df)

            # Add corporate actions
            actions = get_corporate_actions(ticker, start_date, end_date)
            df['Dividends'] = actions['Dividends']
            df['Splits'] = actions['Splits']

            final_data.append(df)

        except Exception as e:
            print(f"Skipped {ticker}: {str(e)}")

    return pd.concat(final_data, ignore_index=True).dropna()

full_dataset = build_full_dataset(dow_list[:3], "2020-01-01", "2023-01-01")  # Test with first 3 tickers
print("\nFinal Dataset Columns:", full_dataset.columns.tolist())
print("\nSample Data:\n", full_dataset.head())

processed_df = build_full_dataset(dow_list, start_date, end_date)

[*********************100%***********************]  1 of 1 completed
<ipython-input-184-16f7ab0c4e8e>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grouped.apply(calculate_indicators)
[*********************100%***********************]  1 of 1 completed
<ipython-input-184-16f7ab0c4e8e>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grouped.apply(calculate_indicators)
[*******************


Final Dataset Columns: ['Date', 'Close', 'High', 'Low', 'Open', 'Volume', 'ticker', 'SMA_20', 'EMA_20', 'MACD', 'Signal_Line', 'RSI', '%K', '%D', 'Middle_Band', 'Upper_Band', 'Lower_Band', 'ATR', 'CMF', 'Volume_ROC', 'Log_Returns', 'Rolling_Vol_21D', 'Drawdown', 'Sharpe', 'Dividends', 'Splits']

Sample Data:
          Date      Close       High        Low       Open     Volume ticker  \
21 2020-02-03  74.727974  75.897335  73.168820  73.672395  173788400   AAPL   
22 2020-02-04  77.194984  77.386249  75.931199  76.337933  136616400   AAPL   
23 2020-02-05  77.824463  78.625828  77.219202  78.325613  118826800   AAPL   
24 2020-02-06  78.734779  78.737203  77.536367  78.095628  105425600   AAPL   
25 2020-02-07  77.664574  78.482401  77.171936  78.232442  117684000   AAPL   

       SMA_20     EMA_20      MACD  ...  Lower_Band       ATR       CMF  \
21  75.792247  75.721162  0.764573  ...   72.320495  1.748168  0.124594   
22  76.022851  75.861526  0.790719  ...   72.848739  1.844662  


<ipython-input-184-16f7ab0c4e8e>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grouped.apply(calculate_indicators)
[*********************100%***********************]  1 of 1 completed
<ipython-input-184-16f7ab0c4e8e>:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  return grouped.apply(calculate_indicators)
[*********************100%***********************]  1 of 1 completed
<ipython-input-184-

## Normalize/Standardize for NN

In [195]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#### a. Min-Max Scaling (0-1 range):

In [196]:
def scale_features_min_max(df):
    """Normalize feature columns using Min-Max scaling"""
    # Columns to exclude from scaling
    non_feature_cols = ['Date', 'ticker']
    feature_cols = [col for col in df.columns if col not in non_feature_cols]

    # Initialize scaler and fit to training data
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_values = scaler.fit_transform(df[feature_cols])

    # Rebuild DataFrame
    scaled_df = pd.DataFrame(scaled_values, columns=feature_cols)
    return pd.concat([df[non_feature_cols], scaled_df], axis=1), scaler

#### b. Standardization (Z-score):

In [205]:
def scale_features_z_score(df, scaler=None):
    """Z-score normalization while preserving market data"""
    # Columns to preserve unchanged
    preserved = [
        'Date', 'ticker', 'Close', 'High', 'Low', 'Open', 'Volume',
        'Log_Returns', 'Dividends', 'Splits', 
        'SMA_20', 'EMA_20', 'Middle_Band', 'Upper_Band', 'Lower_Band',  # Price-based indicators
        'ATR', 'Rolling_Vol_21D', 'Drawdown', 'Sharpe'  # Risk metrics
    ]
    
    # Columns to scale (technical indicators)
    scale_cols = [c for c in df.columns if c not in preserved]
    
    # Separate data
    metadata = df[preserved].copy()
    features = df[scale_cols]
    
    # Initialize/reuse scaler
    if scaler is None:
        scaler = StandardScaler()
        scaled_features = scaler.fit_transform(features)
    else:
        scaled_features = scaler.transform(features)
    
    # Rebuild DataFrame
    scaled_df = pd.DataFrame(scaled_features, 
                            columns=scale_cols, 
                            index=df.index)
    
    return pd.concat([metadata, scaled_df], axis=1), scaler

## Chronological Data Splitting


In [210]:
# Modified time series split to maintain temporal structure
def time_series_split(df, train_ratio=0.7, test_ratio=0.2):
    """Split data sequentially while maintaining time order"""
    df = df.sort_values('Date').reset_index(drop=True)
    n = len(df)
    
    train_end = int(n * train_ratio)
    test_end = train_end + int(n * test_ratio)
    
    return (
        df.iloc[:train_end],  # Train
        df.iloc[train_end:test_end],  # Test
        df.iloc[test_end:]  # Black swan (remaining data)
    )

# Proper usage flow:
# 1. Split first to prevent leakage
train_raw, test_raw, black_swan_raw = time_series_split(processed_df)

# 2. Scale using training data statistics
train_scaled, scaler = scale_features_z_score(train_raw)
test_scaled, _ = scale_features_z_score(test_raw, scaler)
black_swan_scaled, _ = scale_features_z_score(black_swan_raw, scaler)

# 3. Verify critical columns remain unscaled
print(train_scaled[['Date', 'ticker', 'Close', 'Log_Returns']].head())


## Full Pipeline Integration


In [211]:
# 1. Split first to prevent leakage
train_raw, test_raw, black_swan_raw = time_series_split(processed_df)

# 2. Scale using training data statistics
train_scaled, scaler = scale_features_z_score(train_raw)
test_scaled, _ = scale_features_z_score(test_raw, scaler)
black_swan_scaled, _ = scale_features_z_score(black_swan_raw, scaler)

# 3. Verify critical columns remain unscaled
print(train_scaled[['Date', 'ticker', 'Close', 'Log_Returns']].head())

# Final formatted datasets
print(f"Training set shape: {train_scaled.shape}")
print(f"Test set shape: {test_scaled.shape}")
print(f"Black swan reserve shape: {black_swan_scaled.shape}")

# Verify columns
print("\nSample scaled training data:")
print(train_scaled[['Date', 'ticker', 'Close', 'SMA_20']].head())

Training samples: 26901
Test samples: 7686
Black swan samples: 3843


## Export Data as CSV for ML/DL

In [212]:
import os

# Create directory if needed
os.makedirs('data', exist_ok=True)

# Save scaled datasets
train_scaled.to_csv('data/train_scaled.csv', index=False)
test_scaled.to_csv('data/test_scaled.csv', index=False)
black_swan_scaled.to_csv('data/black_swan_scaled.csv', index=False)

# Optional: Save scaler for new data transformations
# from joblib import dump
# dump(scaler, 'data/scaler.joblib')

print("Saved files:")
print(os.listdir('data'))


Saved files:
['test_scaled.csv', 'black_swan_scaled.csv', 'train_scaled.csv']
